### Сторонние библиотеки использовать нельзя

### Задача 0 [Библиотека] (0.15 балла)  

**Условие:** 


В библиотеке хранятся книги и журналы. У каждой сущности есть общие характеристики, такие как: название, автор, жанр, число страниц, формат страниц, индекс редкости (от 1 до 10) и текст. Также у разных сущностей могут быть свои атрибуты. Хочется все редкие издания (индекс 9 или 10) дополнительно сохранять в некое хранилище (пусть json-файл), а также хочется понимать какую площадь занимает издание, если разложить все его страницы на полу.     


**Комментарий:**

Это задача с семинара на организацию иерархии классов. Идея в том, что нужно разделять сущности в зависимости от их применения. Например, есть книга как некий абстрактный объект, а есть библиотечная книга, у которой есть свои особенности. Также для сохранения книг в json нужно использвать классы-примеси.


Иерархия классов:

In [249]:
PAGES_FORMAT = {
    'A1': (2048, 1024),
    'A2': (1024, 512),
    'A3': (512, 256),
    'A4': (297, 210),
    'A5': (210, 148),   
    }

import json
class ReadableEntity:
    
    def __init__(self, name, author, genre, pages_num, page_format, rarity, text):
        
        self.name = name   
        self.author = author
        self.genre = genre
        self.pages_num = pages_num
        self.page_format = page_format
        self.rarity = rarity
        self.text = text
        
    @property
    def page_format(self):
        return self._page_format
    
    @page_format.setter
    def page_format(self, form):
        if form.upper() not in PAGES_FORMAT:
            raise ValueError("Unrecognised page format")
        else:
            self._page_format = form
    
    @property
    def pages_num(self):
        return self._pages_num
    
    @pages_num.setter
    def pages_num(self, num):
        if not isinstance(num, int):
            raise ValueError("Number of pages should be integer")
        if num < 0:
            return ValueError("Number of pages should be greater than 0")
        else:
            self._pages_num = num
            
    @property
    def rarity(self):
        return self._rarity
    
    @rarity.setter
    def rarity(self, value):
        if not isinstance(value, int):
            raise ValueError("Rarity index should be integer")
        if value not in list(range(1, 11)):
            return ValueError("Rarity index should be between 1 and 10")
        else:
            self._rarity = value
    
    @staticmethod
    def _calc_area(heigth, width):
        return heigth * width
    
    @property
    def pages_square(self):
        heigth, width = PAGES_FORMAT[self.page_format.upper()]
        return self.pages_num * self._calc_area(heigth, width)
    
class Journal(ReadableEntity):
    def __init__(self, journal_id, publisher, issue, *kwargs):
        self.journal_id = journal_id # some library identifier
        self.publisher = publisher
        self.issue = issue
        super(ReadableEntity, self).__init__(*kwargs)
        
    @property    
    def journal_id(self):
        return self._journal_id
    
    @journal_id.setter
    def journal_id(self, value): 
        if not isinstance(value, int):
            raise ValueError("Journal id should be integer")
        else:
            self._journal_id = value
        
    @property
    def issue(self):
        return self._issue
    
    @issue.setter
    def issue(self, value):
        if not isinstance(value, int):
            raise ValueError("Issue number should be integer")
        else:
            self._issue = issue


class Book(ReadableEntity):
    def __init__(self, book_id, shelf_no, library_only, *kwargs):
        self.book_id = book_id # some library identifier
        self.shelf_no = shelf_no 
        self.library_only = library_only # can be taken home or not
        super().__init__(*kwargs) 
        
    @property       
    def book_id(self):
        return self._book_id
    
    @book_id.setter
    def book_id(self, value):
        if not isinstance(value, int):
            raise ValueError("Book id should be integer")
        else:
            self._book_id = value
            
    @property
    def shelf_no(self):
        return self._shelf_no
    
    @shelf_no.setter
    def shelf_no(self, value):
        if not isinstance(value, int):
            raise ValueError("Shelf number should be integer")
        else:
            self._shelf_no = value
    
    @property
    def library_only(self):
        return self._library_only
    
    @library_only.setter
    def library_only(self, value):
        if not isinstance(value, bool):
            raise ValueError("No condition specified. Shuld be True/False")
        else:
            self._library_only = value


class Exporter:
    
    def export_to_txt(self, file_path):
        with open(file_path, 'w') as f:
            for key in self.__dict__:
                f.write("{}: {}\n".format(key, self.__dict__[key]))
            f.write("\n")

    def export_to_json(self, file_path):
        with open(file_path, 'w') as f:
            json.dump(self.__dict__, file_path)
    
class LibraryJournal(Journal, Exporter):
    def __init__(self, *kwargs):
        super(LibraryJournal, self).__init__(*kwargs) 
        RARE_RATE = [9, 10]
        WRITE_PATH = './rare_journals/'
        if self.rarity in rare_rate:
            path = write_path + str(self.journal_id) + '.json'
            export_to_json(self, path)
            print('Rare journal written to json')

class LibraryBook(Book, Exporter):
    def __init__(self, *kwargs):
        super(LibraryBook, self).__init__(*kwargs) 
        RARE_RATE = [9, 10]
        WRITE_PATH = './rare_books/'
        if self.rarity in RARE_RATE:
            path = WRITE_PATH + str(self.book_id) + '.json'
            export_to_json(self, path)
            print('Rare book written to json')

#### Пример обычной книги:

In [250]:
name = 'Война и Мир'
author = 'Толстой Лев Николаевич'
genre = 'Роман'
pages_num = 1143
page_format = 'A5'
rarity = 2
text = '— Eh bien, mon prince. Gênes et Lucques ne sont plus que des apanages,...'

In [251]:
SomeBook = ReadableEntity(name, author, genre, pages_num, page_format, rarity, text)

In [252]:
SomeBook.pages_square

35524440

#### Делаем ее библиотечной:

In [253]:
book_id = 543
shelf_no = 23
library_only = False

In [279]:
LibBook = LibraryBook(123, 432, True, name, author, genre, pages_num, page_format, 9, text)\

Rare book written to json


#### То же самое с журналами.

### Задача 1 [Размер объектов] (0 - 0.15 балла)  

**Условие:** 

Написать функцию получения реального объема занимаемой объектом памяти объектом. 


1) Для int, str, list, tuple, dict **(0.05 балла)**

2) Для всех типов **(+0.1 балла)**


**Комментарий:**

На занятиях не раз говорилось, что `sys.getsizeof` умеет находить размер простых объектов, но если речь идет об объектах, вроде list, то функция вернет не совсем то, что может ожидать разработчик, потому что список хранит указатели на объекты. 

*Пример:*
```
sys.getsizeof([]) == 64
sys.getsizeof(['aaaaaaa']) == 72
```
Но
```
sys.getsizeof('aaaaaaa') == 56
```


### Задача 2 [Многочлены] (0.64 балла)

**Условие:**

Реализовать класс многочлена. Определить операции:

1) *сложения* - **(0.02 балла)** 

2) *вычитания* - **(0.02 балла)**

3) *умножения* - **(0.04 балла)** 

3a) *быстрого умножения* (алгоритм Карацубы или быстрое преобразование Фурье) - **(+0.25 балла)**

4) *деления* - **(0.05 балла)** 

5) *возведения в степень* - **(0.02 балла)** + | *возведения в степень* через быстрое возведение в степень за log - **(0.04 балла)**

6) *представления многочлена в человеческом виде* - **(0.02 балла)** 

7) *дифференцирования* - **(0.05 балла)** 

8) *интегрирования* - **(0.05 балла)**

9) Вызова многочлена как функции (вычисление значения в точке) - **(0.03 балла)** 

**Комментарии:**

Для комплексных коэффициентов **(0.01 балла)** к каждому пункту.

Операции с числами также должны работать.

In [264]:
class Polynomial(object):
    """
    This class is for handling polynomials.
    
    Can be initialised multiple ways:
        Polynomial(): empty polynomial, just 0
        Polynomial([5]): simple number
        Polynomial([1, 4]): polynomial of degree 1 with coefficients. Like 1+4x^2
        
    Complex coefficients are also supported.
    """
    
    def __init__(self, coeffs): 
        
        'Determines and checks the coefficients'
        
        if list(coeffs):
            for coef in coeffs:
                if isinstance(coef, (int, float, complex)) and not isinstance(coef, bool):
                    pass
                else:
                    raise ValueError('You passed non-numeric type of data')
        else:
            coeffs = [0]
        self.coeffs = coeffs
        
    @staticmethod
    def _format_coeff(coeff):
        if isinstance(coeff, complex):
            return "+{0}".format(coeff)
        else:
            return str(coeff) if coeff < 0 else "+{0}".format(coeff)

    @staticmethod
    def _format_pow(power):
        return 'x^{0}'.format(power) if power != 0 else ''
        
    def __str__(self): 
        
        'String representation of the polynom'
        
        expr = []
        for coeff, power in zip(self.coeffs, range(0, len(self.coeffs))):
            if coeff == 0:
                continue
            expr.append(self._format_coeff(coeff))
            expr.append(self._format_pow(power))
        expr[0] = expr[0].lstrip("+")
        return ''.join(expr)
    
    def __repr__(self):
        
        'Polynom class representanion'
        
        return "%s(%s)" % (type(self).__name__, str(list(self.coeffs)))
    
    def __getitem__(self, index):
        
        'Get coefficient by index'
        
        try:
            return self.coeffs[index]
        except IndexError:
            return 0
        
    def __setitem__(self, index, value):
        
        'Set coefficient by index. Supports non-existent cofficient indices'
        
        try:
            self.coeffs[index] = value
        except IndexError:
            new_coeffs = np.append(self.coeffs, np.zeros(ind - len(self.coeffs) + 1))
            new_coeffs[index] = value
            self.coeffs = newcoeffs
            
    def length(self):
        
        'Actual length of the polynom, degree + 1'
        
        for index, coeff in enumerate(list(self.coeffs)[::-1]):
            if coeff != 0:
                break
        return len(self.coeffs) - index
    
    def degree(self):
        
        'Degree of the polynom'
        
        return self.length() - 1
    
    def __add__(self, polyn):
        
        'Addition operator for polynoms'
        
        max_len = max(self.length(), polyn.length())
        new_coeffs = []
        for index in range(max_len):
            new_coeffs.append(self[index] + polyn[index])
        return Polynomial(new_coeffs)
    
        __radd__ = __add__
    
    def __neg__(self):
        
        '- Polynom'
        
        return Polynomial([-coef for coef in self.coeffs])
    
    def __pos__(self):
        
        '+ Polynom'
        return Polynomial(self.coeffs)
    
    def __sub__(self, polyn):
                
        'Substraction operator for polynoms'
        
        return self + (-polyn)
    
    @staticmethod
    def _fast_mult(x, y):
        
        'Fast multiplication based on Karatsuba algorithm'
        
        if (isinstance(x, complex)) | (isinstance(y, complex)):
            return x * y
        
        elif x < 10 or y < 10:
                return x * y

        n = max(len(str(x)), len(str(y))) // 2

        cut = pow(10, n)
        a, b = x // cut, x % cut
        c, d = y // cut, y % cut

        k0 = fast_mult(a, c)
        k1 = fast_mult((a + b), (c + d))
        k2 = fast_mult(b, d)

        return k0 * pow(10, 2*n) + (k1-k0-k2) * pow(10, n) + k2

    def __mul__(self, polyn):
        
        'Multiplication of 2 polynoms. Supports passing just a numeric value'
        
        if isinstance(polyn, (int, float, complex)):
            polyn = Polynomial([polyn])
        fin_len = self.length() + polyn.length()
        new_coeffs = [sum(self._fast_mult(self[j], polyn[i-j]) for j in range(i+1)) for i in range(fin_len-1)]
        return Polynomial(new_coeffs)

        __rmul__ = __mul__
     
    def __pow__(self, n):
        
        'Power function'
        
        result = Polynomial([1])
        for i in range(n):
            result *= self
        return result
    
    def diff(self):
        
        'Differentiation function'
        
        result = []
        for ind, coef in enumerate(self.coeffs):
            result.append(ind * coef)
        return Polynomial(result[1:])
    
    def integral(self):
        
        'Integration function'
        
        result = [0]
        for ind, coef in enumerate(self.coeffs):
            result.append(coef / (ind + 1))
        return Polynomial(result)
    
    def fit(self, X):
        
        'Calculating polynom value in point X'
        
        if not isinstance(X, (int, float, complex)):
            raise ValueError("X must be numeric")
        else:
            result = 0
            for ind, coef in enumerate(self.coeffs):
                result += coef * X**ind
            return result

### Примеры использования

In [265]:
X = Polynomial([1, 3, 7])
Y = Polynomial([2, 5, 3, 5])

In [266]:
print(X)

1+3x^1+7x^2


In [267]:
X + Y

Polynomial([3, 8, 10, 5])

In [268]:
X - Y

Polynomial([-1, -2, 4, -5])

In [269]:
X * Y

Polynomial([2, 11, 32, 49, 36, 35])

In [270]:
X**2

Polynomial([1, 6, 23, 42, 49])

In [271]:
X.diff()

Polynomial([3, 14])

In [272]:
X.integral()

Polynomial([0, 1.0, 1.5, 2.3333333333333335])

In [273]:
X.fit(10)

731

### Задача 3 [Аналог range] (0.05 балла)

**Условие:**

Реализуйте итератор с поведением, аналогичным range.

In [274]:
class my_range:

    """Iterator that acts exactly like range() function"""
    

    def __init__(self, *args, **kwargs):
        
        if len(args) == 1:
            self.start = 0
            self.stop = args[0]
            self.step = 1
        elif len(args) == 2:
            self.start = args[0]
            self.stop = args[1]
            self.step = 1
        elif len(args) == 3:
            self.start = args[0]
            self.stop = args[1]
            self.step = args[1]
        else:
            raise TypeError('my_range expected from 1 to 3 arguments, got {}'.format(len(args)))
            
        if self.step == 0:
            raise ValueError('Step can not be 0')
        else:
            self.current = self.start - self.step

    def __iter__(self):
        return self

    def __next__(self):
        self.next = self.current + self.step
        if self.step > 0:
            if self.next <= self.stop:
                self.current = self.next
                return self.next
            else:
                 raise StopIteration
        else:
            if self.next >= self.stop:
                self.current = self.next
                return self.next
            else:
                 raise StopIteration

In [276]:
for i in range(5):
    print (i)

0
1
2
3
4


In [277]:
for i in range(5, 8):
    print (i)

5
6
7


In [278]:
for i in range(5, 8, 2):
    print (i)

5
7


### Задача 4 [Primary Key] (0.05 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте Primary Key - свойства первичного ключа из PostgreSQL.

### Задача 5 [PositiveSmallIntegerField] (0.03 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте тип данных PositiveSmallIntegerField - поле, принимающее значения от 0 до 32767.

### Задача 6 [Timer] (0.02 балла)

**Условие:**

Реализовать контекстный менеджер, который выводит время, проведенное в нём.

In [7]:
from time import time
import numpy as np

In [32]:
class Timer(object):
    def __enter__(self):
        self.start = time()
    def __exit__(self, type, value, traceback):
        self.end = time()
        print('time spent: {}'.format(self.end - self.start))

In [33]:
with Timer():
    s = np.random.randint(0, 5000)

time spent: 2.193450927734375e-05


In [34]:
with Timer():
    pass

time spent: 9.5367431640625e-07
